In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import hdbscan
from sklearn.preprocessing import StandardScaler

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/r-maSj5Yegvw2sJraT15FA/ODCAF-v1-0.csv'
df = pd.read_csv(url, encoding = "ISO-8859-1")
print(df.head())

ModuleNotFoundError: No module named 'hdbscan'

In [ ]:
print(df.ODCAF_Facility_Type.value_counts())

ODCAF_Facility_Type
library or archives                     3013
museum                                  1938
gallery                                  810
heritage or historic site                620
theatre/performance and concert hall     583
festival site                            346
miscellaneous                            343
art or cultural centre                   225
artist                                    94
Name: count, dtype: int64


In [ ]:
df=df[df.ODCAF_Facility_Type =="museum"]
print(df.ODCAF_Facility_Type.value_counts())


ODCAF_Facility_Type
museum    1938
Name: count, dtype: int64


In [ ]:
print(df.columns)
df=df[["Latitude","Longitude"]]
print(df)
print(df.info())
print(df.isnull().sum())
df = df[df.Latitude!='..']
df[["Latitude","Longitude"]]=df[["Latitude","Longitude"]].astype('float')

Index(['Index', 'Facility_Name', 'Source_Facility_Type', 'ODCAF_Facility_Type',
       'Provider', 'Unit', 'Street_No', 'Street_Name', 'Postal_Code', 'City',
       'Prov_Terr', 'Source_Format_Address', 'CSD_Name', 'CSDUID', 'PRUID',
       'Latitude', 'Longitude'],
      dtype='object')
         Latitude     Longitude
1      55.2645508  -127.6428124
2       45.963283   -66.6419017
8      49.1763542   -123.112783
13      49.261938   -123.151123
15    49.88955855  -97.23574396
...           ...           ...
7940    43.857692   -79.3619396
7954           ..            ..
7958           ..            ..
7968  48.42241956  -123.3435527
7969   48.4260053  -123.3691883

[1938 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 1938 entries, 1 to 7969
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Latitude   1938 non-null   object
 1   Longitude  1938 non-null   object
dtypes: object(2)
memory usage: 45.4+ KB
None

In [ ]:
# In this case we know how to scale the coordinates. Using standardization would be an error becaues we aren't using the full range of the lat/lng coordinates.
# Since latitude has a range of +/- 90 degrees and longitude ranges from 0 to 360 degrees, the correct scaling is to double the longitude coordinates (or half the Latitudes)
coords_scaled = df.copy()
coords_scaled["Latitude"] = 2*coords_scaled["Latitude"]
print(coords_scaled)

        Latitude   Longitude
1     110.529102 -127.642812
2      91.926566  -66.641902
8      98.352708 -123.112783
13     98.523876 -123.151123
15     99.779117  -97.235744
...          ...         ...
7934   86.366180  -79.224564
7936   87.380043  -79.476208
7940   87.715384  -79.361940
7968   96.844839 -123.343553
7969   96.852011 -123.369188

[1607 rows x 2 columns]


In [ ]:
N=4 # minimum number of samples needed to form a neighbourhood
eps=1.0 # neighbourhood search radius
metric='euclidean' # distance measure 

dbscan = DBSCAN(eps=eps, min_samples=N, metric=metric).fit(coords_scaled)
df['Cluster'] = dbscan.fit_predict(coords_scaled)  # Assign the cluster labels

# Display the size of each cluster
df['Cluster'].value_counts()

NameError: name 'DBSCAN' is not defined

In [ ]:
min_samples=None
min_cluster_size=3
hdb = hdbscan.HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size, metric='euclidean')  # You can adjust parameters as needed
# Assign labels
df['Cluster'] = hdb.fit_predict(coords_scaled)  # Another way to assign the labels

# Display the size of each cluster
df['Cluster'].value_counts()
plot_clustered_locations(df, title='Museums Hierarchically Clustered by Proximity')